In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
%pip install haversine
from haversine import haversine

In [0]:
import pandas as pd
latlonData = pd.read_csv('/content/drive/My Drive/AI Assignment/nodes.csv')

def calculateHeuristic(currNode):
  [currLat,currLon] = latlonData[latlonData['id']==currNode].iloc[0][['lat','lon']]
  curr = (currLat, currLon)
  dest = (17.240673, 78.432342)

  return haversine(curr, dest)

In [5]:
%pip install networkx
import networkx as nx

graphData = pd.read_csv('/content/drive/My Drive/AI Assignment/edges.csv')
graphData = graphData[['source','target','length']]

graphType = nx.Graph()
g = nx.from_pandas_edgelist(graphData, edge_attr='length', create_using=graphType)

import heapq

def createPath(cameFrom, current):
  path = [current]
  while current in cameFrom.keys():
    current = cameFrom[current]
    path.insert(0,current)
  return path


def aStar(srcNode, destNode):
    openSet = []
    
    cameFrom = {}
    cameFrom[srcNode] = None

    cost = {}
    cost[srcNode] = 0
    
    fScore = {}
    fScore[srcNode] = 0

    gScore = {}
    gScore[srcNode] = 0

    heapq.heappush(openSet, (srcNode, fScore))
    
    while len(openSet) > 0:
      
      currentNode = heapq.heappop(openSet)
      
      # if true then create path and return it
      if (currentNode[0] == destNode):
        return createPath(cameFrom,currentNode[0])
        
      #fetch neighbours of current node
      neighbourData = list(g.neighbors(currentNode[0]))
        
      #in neighbourData
      for item in neighbourData:
        #distance = length of edge between current & neighbourNode
        #g_Score = 0 if newly visited node or has some value assosiated to it calculated previously
        #neighbourNode is osm id of the node
        neighbourNode = item
        distance = g[currentNode[0]][neighbourNode]['length']
        if neighbourNode not in cameFrom:
          # cost = distance from srcNode to neighbourNode through currentNode
          cost[neighbourNode] = gScore[currentNode[0]] + distance
          # if we have a neighbour that has been visited already by some other path
          # and we are visiting it again via new path, we check which one is optimum
          if cost[neighbourNode] < gScore.get(neighbourNode, float('inf')):
            cameFrom[neighbourNode] = currentNode[0]
            gScore[neighbourNode] = cost[neighbourNode]
            fScore[neighbourNode] = gScore[neighbourNode] + calculateHeuristic(neighbourNode)
            if neighbourNode not in openSet:
              heapq.heappush(openSet, (neighbourNode, fScore))
    return openSet

In [0]:
srcNode = 7065632060
destNode = 5711258337

route = aStar(srcNode, destNode)
route.pop(0)

if len(route)==0:
  print(f'Fatal Error: Path doesn\'t exist')

In [0]:
latlonRoute = []
for node in route:
  [lat,lon] = latlonData[latlonData['id']==node].iloc[0][['lat','lon']]
  latlonRoute.append((lat,lon))
latlonRoute

from statistics import mean
meanLat = mean(point[0] for point in latlonRoute)
meanLon = mean(point[1] for point in latlonRoute)

#%pip install gmplot
import gmplot

routeLats, routeLons = zip(*latlonRoute)
gmap3 = gmplot.GoogleMapPlotter(meanLat, meanLon, 13)
#gmap3.scatter(routeLats, routeLons, '#FF0000', size = 50, marker = False )
gmap3.plot(routeLats, routeLons, 'cornflowerblue', edge_width = 3.0)
gmap3.draw('route.html')